In [1]:
import requests
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import csv
import time
import json
from pandas import json_normalize

from sklearn.decomposition import PCA

In [2]:
train = pd.read_csv('data/1차원트레인.csv', encoding='cp949')
test = pd.read_csv('data/1차원테스트.csv', encoding='cp949')
gender = pd.read_csv('data/age_gender_info.csv')

display(train)
display(test)
display(gender)

,Unnamed: 0,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,신분,임대보증금,...,면적_25.0,면적_70.0,면적_80.0,면적_15.0,면적_10.0,면적_20.0,면적_75.0,면적_60.0,면적_65.0,등록차량수
0,0,C2483,900.0,0.0,0.0,0.0,35.0,134.0,0.0,15667000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1015.0
1,1,C2371,474.0,0.0,0.0,0.0,35.0,42.0,0.0,9731000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,451.0
2,2,C2097,998.0,0.0,0.0,0.0,30.0,184.0,0.0,13158000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,995.0
3,3,C1368,896.0,0.0,0.0,0.0,35.0,176.0,0.0,13731000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,861.0
4,4,C1057,521.0,0.0,0.0,0.0,35.0,134.0,0.0,13731000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,521.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,418,C1652,890.0,0.0,14.0,2.0,15.0,188.0,3.0,40400000.0,...,336.0,0.0,0.0,188.0,0.0,0.0,0.0,0.0,0.0,509.0
419,419,C2051,1164.0,0.0,15.0,1.0,55.0,2.0,0.0,39000000.0,...,0.0,331.0,470.0,0.0,0.0,0.0,45.0,0.0,24.0,780.0
420,420,C2612,1684.0,0.0,15.0,0.0,25.0,628.0,0.0,9800000.0,...,682.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1832.0
421,421,C1802,1438.0,0.0,15.0,1.0,55.0,32.0,0.0,39000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1983.0


,Unnamed: 0,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,신분,임대보증금,...,면적_40.0,면적_25.0,면적_70.0,면적_80.0,면적_15.0,면적_10.0,면적_20.0,면적_75.0,면적_60.0,면적_65.0
0,0,C1072,754.0,0.0,3.0,0.0,35.0,116.0,2.0,22830000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,C1128,1354.0,0.0,3.0,0.0,35.0,368.0,2.0,22830000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,C1332,1297.0,0.0,3.0,0.0,35.0,282.0,2.0,28598000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,C1563,1974.0,0.0,3.0,0.0,35.0,214.0,0.0,30040000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,C1721,663.0,0.0,3.0,0.0,35.0,155.0,0.0,26435000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,145,C1633,178.0,0.0,13.0,0.0,25.0,88.0,0.0,10305000.0,...,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,146,C2451,395.0,0.0,13.0,1.0,55.0,23.0,0.0,50000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,147,C2292,906.0,0.0,15.0,0.0,20.0,336.0,2.0,8738000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,336.0,0.0,0.0,0.0
148,148,C2551,674.0,0.0,15.0,0.0,20.0,160.0,2.0,8768000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,352.0,0.0,0.0,0.0


,지역,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,경상북도,0.030158,0.033195,0.056346,0.061360,0.060096,0.067859,0.053433,0.049572,0.083660,...,0.082684,0.063889,0.047717,0.030172,0.029361,0.011211,0.005578,0.001553,0.000234,0.000014
1,경상남도,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,...,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
2,대전광역시,0.028197,0.029092,0.040490,0.042793,0.060834,0.064247,0.068654,0.066848,0.074667,...,0.088468,0.070261,0.051010,0.037143,0.032455,0.013751,0.006494,0.001740,0.000298,0.000066
3,경기도,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,...,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
4,전라북도,0.028089,0.029065,0.059685,0.060080,0.066262,0.070322,0.052027,0.046596,0.077005,...,0.076636,0.068042,0.051025,0.035748,0.035049,0.012641,0.007223,0.001898,0.000158,0.000013
5,강원도,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,...,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
6,광주광역시,0.031994,0.034566,0.049851,0.050254,0.065084,0.066875,0.066888,0.064416,0.080028,...,0.075930,0.059586,0.048552,0.031754,0.029749,0.010341,0.006343,0.000895,0.000353,0.000013
7,충청남도,0.031369,0.031711,0.059077,0.062422,0.067975,0.072622,0.065095,0.067303,0.078860,...,0.070278,0.057692,0.042296,0.028682,0.024514,0.010900,0.005429,0.001549,0.000219,0.000123
8,부산광역시,0.022003,0.022947,0.032681,0.035512,0.053796,0.057233,0.047049,0.048866,0.061952,...,0.109297,0.085294,0.078743,0.053388,0.047908,0.020228,0.008043,0.002240,0.000268,0.000028
9,제주특별자치도,0.034690,0.036695,0.060094,0.063080,0.069135,0.069667,0.050808,0.048026,0.080548,...,0.074248,0.055717,0.047944,0.033054,0.026836,0.011332,0.006832,0.000982,0.000368,0.000082


In [3]:
gender.replace('경상북도', 0 , inplace=True)
gender.replace('경상남도', 1 , inplace=True)
gender.replace('대전광역시', 2 , inplace=True)
gender.replace('경기도', 3 , inplace=True)
gender.replace('전라북도', 4 , inplace=True)
gender.replace('강원도', 5 , inplace=True)
gender.replace('광주광역시', 6 , inplace=True)
gender.replace('충청남도', 7 , inplace=True)
gender.replace('부산광역시', 8 , inplace=True)
gender.replace('제주특별자치도', 9 , inplace=True)
gender.replace('울산광역시', 10 , inplace=True)
gender.replace('충청북도', 11 , inplace=True)
gender.replace('전라남도', 12 , inplace=True)
gender.replace('대구광역시', 13 , inplace=True)
gender.replace('서울특별시', 14 , inplace=True)
gender.replace('세종특별자치시', 15, inplace=True)
gender

,지역,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,0,0.030158,0.033195,0.056346,0.061360,0.060096,0.067859,0.053433,0.049572,0.083660,...,0.082684,0.063889,0.047717,0.030172,0.029361,0.011211,0.005578,0.001553,0.000234,0.000014
1,1,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,...,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
2,2,0.028197,0.029092,0.040490,0.042793,0.060834,0.064247,0.068654,0.066848,0.074667,...,0.088468,0.070261,0.051010,0.037143,0.032455,0.013751,0.006494,0.001740,0.000298,0.000066
3,3,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,...,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
4,4,0.028089,0.029065,0.059685,0.060080,0.066262,0.070322,0.052027,0.046596,0.077005,...,0.076636,0.068042,0.051025,0.035748,0.035049,0.012641,0.007223,0.001898,0.000158,0.000013
5,5,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,...,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
6,6,0.031994,0.034566,0.049851,0.050254,0.065084,0.066875,0.066888,0.064416,0.080028,...,0.075930,0.059586,0.048552,0.031754,0.029749,0.010341,0.006343,0.000895,0.000353,0.000013
7,7,0.031369,0.031711,0.059077,0.062422,0.067975,0.072622,0.065095,0.067303,0.078860,...,0.070278,0.057692,0.042296,0.028682,0.024514,0.010900,0.005429,0.001549,0.000219,0.000123
8,8,0.022003,0.022947,0.032681,0.035512,0.053796,0.057233,0.047049,0.048866,0.061952,...,0.109297,0.085294,0.078743,0.053388,0.047908,0.020228,0.008043,0.002240,0.000268,0.000028
9,9,0.034690,0.036695,0.060094,0.063080,0.069135,0.069667,0.050808,0.048026,0.080548,...,0.074248,0.055717,0.047944,0.033054,0.026836,0.011332,0.006832,0.000982,0.000368,0.000082


In [4]:
등록차량수 = train.iloc[:,-1]
train = train.drop('등록차량수', axis=1)

res = pd.merge(train, gender, on='지역')
res1 = pd.merge(test, gender, on='지역')

# res = train
# res1 = test

display(res)
display(res1)

,Unnamed: 0,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,신분,임대보증금,...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,0,C2483,900.0,0.0,0.0,0.0,35.0,134.0,0.0,15667000.0,...,0.082684,0.063889,0.047717,0.030172,0.029361,0.011211,0.005578,0.001553,0.000234,0.000014
1,1,C2371,474.0,0.0,0.0,0.0,35.0,42.0,0.0,9731000.0,...,0.082684,0.063889,0.047717,0.030172,0.029361,0.011211,0.005578,0.001553,0.000234,0.000014
2,2,C2097,998.0,0.0,0.0,0.0,30.0,184.0,0.0,13158000.0,...,0.082684,0.063889,0.047717,0.030172,0.029361,0.011211,0.005578,0.001553,0.000234,0.000014
3,3,C1368,896.0,0.0,0.0,0.0,35.0,176.0,0.0,13731000.0,...,0.082684,0.063889,0.047717,0.030172,0.029361,0.011211,0.005578,0.001553,0.000234,0.000014
4,4,C1057,521.0,0.0,0.0,0.0,35.0,134.0,0.0,13731000.0,...,0.082684,0.063889,0.047717,0.030172,0.029361,0.011211,0.005578,0.001553,0.000234,0.000014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,418,C1652,890.0,0.0,14.0,2.0,15.0,188.0,3.0,40400000.0,...,0.119639,0.078939,0.097543,0.061050,0.047853,0.023463,0.011344,0.003326,0.000532,0.000152
419,419,C2051,1164.0,0.0,15.0,1.0,55.0,2.0,0.0,39000000.0,...,0.065279,0.051163,0.026056,0.026223,0.013893,0.007086,0.003459,0.001004,0.000279,0.000112
420,420,C2612,1684.0,0.0,15.0,0.0,25.0,628.0,0.0,9800000.0,...,0.065279,0.051163,0.026056,0.026223,0.013893,0.007086,0.003459,0.001004,0.000279,0.000112
421,421,C1802,1438.0,0.0,15.0,1.0,55.0,32.0,0.0,39000000.0,...,0.065279,0.051163,0.026056,0.026223,0.013893,0.007086,0.003459,0.001004,0.000279,0.000112


,Unnamed: 0,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,신분,임대보증금,...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,0,C1072,754.0,0.0,3.0,0.0,35.0,116.0,2.0,22830000.0,...,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
1,1,C1128,1354.0,0.0,3.0,0.0,35.0,368.0,2.0,22830000.0,...,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
2,2,C1332,1297.0,0.0,3.0,0.0,35.0,282.0,2.0,28598000.0,...,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
3,3,C1563,1974.0,0.0,3.0,0.0,35.0,214.0,0.0,30040000.0,...,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
4,4,C1721,663.0,0.0,3.0,0.0,35.0,155.0,0.0,26435000.0,...,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,145,C1633,178.0,0.0,13.0,0.0,25.0,88.0,0.0,10305000.0,...,0.094511,0.073133,0.055914,0.036572,0.031887,0.012556,0.005364,0.001259,0.000219,0.000000
146,146,C2451,395.0,0.0,13.0,1.0,55.0,23.0,0.0,50000000.0,...,0.094511,0.073133,0.055914,0.036572,0.031887,0.012556,0.005364,0.001259,0.000219,0.000000
147,147,C2292,906.0,0.0,15.0,0.0,20.0,336.0,2.0,8738000.0,...,0.065279,0.051163,0.026056,0.026223,0.013893,0.007086,0.003459,0.001004,0.000279,0.000112
148,148,C2551,674.0,0.0,15.0,0.0,20.0,160.0,2.0,8768000.0,...,0.065279,0.051163,0.026056,0.026223,0.013893,0.007086,0.003459,0.001004,0.000279,0.000112


In [5]:
res = res.join(등록차량수)

# res['총세대수'] = res['총세대수'] - res['공가수']
# res = res.drop('공가수', axis=1)
# display(res)

# res1['총세대수'] = res1['총세대수'] - res1['공가수']
# res1 = res1.drop('공가수', axis=1)
# display(res1)

res.to_csv('data/train_gender.csv', encoding='cp949')
res1.to_csv('data/test_gender.csv', encoding='cp949')